In [23]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"

s3_bucket = "s3://mist-secorapp-{env}/rrm-local/rrm-local-{env}/".format(env=env)

date_day = "2020-03-0[45]"
hr = '*'

rrm_local_path = "dt={day}/hr={hr}/*.seq".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-secorapp-production/rrm-local/rrm-local-production/dt=2020-03-0[45]/hr=*/*.seq


In [11]:
rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)


In [14]:
rdd_rrm_local.first()

(59408017,
 bytearray(b'{"updated": true, "clients": 2, "pre_bandwidth": 20, "power": 8, "timestamp": 1583280001, "site_id": "5dab2c3d-959c-460a-b032-3370d5bae2fb", "org_id": "bbb101eb-b62d-4fb1-8c3d-030c6db7e208", "band": "24", "bandwidth": 20, "rad_id": "r24", "rrm_bandwidth": 20, "rrm_power": 8, "rrm_channel": 6, "pre_power": 8, "when": "2020-03-04T00:00:01", "rrm_enabled": true, "reason": "non-wifi", "ap_id": "5c5b354e3741", "source_data": "random.no_rrm_channel", "command": "capacity-anomaly", "channel_update_interval": 7200, "pre_channel": 11, "channel": 6}'))

In [26]:
df_rrm = rdd_rrm_local.map(lambda x: json.loads(x[1])).toDF()
df_rrm.show()

/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+------------+----+---------+-------+-----------------------+-------+--------------------+--------------------+-----+-------------+-----------+---------+------+--------------+-------------+-----------+-----------+---------+--------------------+--------------------+----------+-------+-------------------+
|       ap_id|band|bandwidth|channel|channel_update_interval|clients|             command|              org_id|power|pre_bandwidth|pre_channel|pre_power|rad_id|        reason|rrm_bandwidth|rrm_channel|rrm_enabled|rrm_power|             site_id|         source_data| timestamp|updated|               when|
+------------+----+---------+-------+-----------------------+-------+--------------------+--------------------+-----+-------------+-----------+---------+------+--------------+-------------+-----------+-----------+---------+--------------------+--------------------+----------+-------+-------------------+
|5c5b354e3741|  24|       20|      6|                   7200|      2|    capacity-ano

In [29]:
df_USNYELE = df_rrm.filter(col('site_id')==user_site_USNYELE)
df_USFLLON = df_rrm.filter(col('site_id')==user_site_USFLLON)

In [33]:
df_USNYELE.select("command", "reason").groupBy('reason').count().show()

+--------------------+-----+
|              reason|count|
+--------------------+-----+
|            non-wifi|   19|
|wifi-cochannel-ex...|    2|
|      rrmG-scheduled|   35|
|      wifi-cochannel|    8|
+--------------------+-----+



In [ ]:
df_USFLLON.select("command", "reason").groupBy('reason').count().show()